In [ ]:
from jdaviz import Imviz, enable_hot_reloading

enable_hot_reloading()

viz = Imviz()

viz.load_data('mast:JWST/product/jw02727-o002_t062_nircam_clear-f090w_i2d.fits', cache=True)

viz.show('sidecar:split-bottom')

This creates a jdaviz-compatible `Viewer` that can appear alongside the jdaviz image viewer: 

In [ ]:
from glue_jupyter.view import IPyWidgetView

class MastAladinView(IPyWidgetView):
    inherit_tools = False
    _toolbar_cls = None
    _figure_widget = None
    _deprecated_data_menu = None
    data_menu = None
    _layer_artist_container = []
    tools = []
    
    @property
    def figure_widget(self):
        return self._figure_widget

    @figure_widget.setter
    def figure_widget(self, widget):
        self._figure_widget = widget

    def initialize_toolbar(self):
        pass

    def _deprecated_data_menu(self):
        pass


demo_view = MastAladinView(viz.app.session)

Here we initialize the MastAladin widget, and we attach style options to it directly. this allows us to see the outermost div with a solid red border.

**The cell below will fail if you run it before the imviz output is visible.**

In [ ]:
from mast_aladin_lite import MastAladin
from ipywidgets import VBox, Layout

# here we explicitly ask to create the mast aladin widget
# with the same height as the already-loaded image viewer
# in imviz. we add the height of the toolbar (seen in the
# image viewer but not in the aladin viewer) to the height 
# of the image-viewer's image for the expected height of the 
# aladin viewer:

# toolbar height is hard coded in jdaviz here:
# https://github.com/spacetelescope/jdaviz/blob/a3588cf43f3e68fde86688088f4578004be4aa39/jdaviz/main_styles.vue#L193
toolbar_height = 42  

height = viz.default_viewer._obj.figure_widget.marks[0].shape[0] + toolbar_height
mast_aladin = MastAladin(height=height)

# mast_aladin.layout = Layout(height='inherit', border="3px solid red")

demo_view.figure_widget = mast_aladin

In [ ]:
height

Now let's put the viewer in jdaviz:

In [ ]:
import uuid

name = 'aladin-lite'

# add viewer to the viewer store:
viz.app._viewer_store[name] = demo_view

# create a viewer item that refers to the MastAladin widget:
viewer_item = {
    'id': name,
    'name': name,    
    'widget': "IPY_MODEL_" + demo_view.figure_widget.model_id,
    'reference': None,
}


# create a stack item, which creates the outer 
stack_item = {
    'id': str(uuid.uuid4()),
    'container': 'gl-stack',
    'children': [],
    'viewers': [viewer_item]
}


viz.app.state.stack_items.append(stack_item)